## Seattle Air BnB Analysis

Below I aim to analysis the Seattle Air BNB Data and answer 3 questions:


I import all necesary packages below, such as pandas, numpy seaborn, sckit-learn etc!

In [1]:
import warnings
warnings.filterwarnings("ignore")

# Libraries for data manipulation and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

# Algorithms to use
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

# Metrics to evaluate the model
from sklearn.metrics import confusion_matrix, classification_report,f1_score
from sklearn import metrics

# For hyperparameter tuning
from sklearn.model_selection import GridSearchCV

Let's read in the first csv entilted "calendar"!

In [2]:
calendar = pd.read_csv('calendar.csv')
calendar.head()

,listing_id,date,available,price
0,241032,2016-01-04,t,$85.00
1,241032,2016-01-05,t,$85.00
2,241032,2016-01-06,f,NaN
3,241032,2016-01-07,f,NaN
4,241032,2016-01-08,f,NaN


In [3]:
calendar.shape

(1393570, 4)

In [4]:
calendar.isnull().sum()/(calendar.isnull().sum() + calendar.notnull().sum())

listing_id    0.00000
date          0.00000
available     0.00000
price         0.32939
dtype: float64

Here we can see that 32% of the values are missing for price -- I would argue to keep the variable given that we still have 68% of it so its a majority.  The other columns are completely filled!

I think it would be good to get the price for each listing id, so I am going to clean the price column so I can make it an int, and then group by the listing_id to get the average price per listing id.

In [5]:
calendar['price']=  calendar['price'].str.replace('$', '', regex=True)
calendar['price']=  calendar['price'].str.replace(',', '', regex=True)
calendar['price']

0          85.00
1          85.00
2            NaN
3            NaN
4            NaN
           ...  
1393565      NaN
1393566      NaN
1393567      NaN
1393568      NaN
1393569      NaN
Name: price, Length: 1393570, dtype: object

In [6]:
calendar['price'] = calendar['price'].fillna(0)

In [7]:
calendar['price']= calendar['price'].astype(float)
calendar['price']= calendar['price'].astype(int)

In [8]:
calendar['price']

0          85
1          85
2           0
3           0
4           0
           ..
1393565     0
1393566     0
1393567     0
1393568     0
1393569     0
Name: price, Length: 1393570, dtype: int64

I want to get the mean price per listing so I group by 'listing_id' below with a mean price.

In [42]:
ind_listing = calendar.groupby('listing_id')['price'].mean()
ind_listing = ind_listing.to_frame()

I have created a new condensed data frame that gives the average price per listing id! 
Next let's read in listings! and then reviews!

In [10]:
listings = pd.read_csv('listings.csv')
listings.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,241032,https://www.airbnb.com/rooms/241032,20160104002432,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,...,10.0,f,NaN,WASHINGTON,f,moderate,f,f,2,4.07
1,953595,https://www.airbnb.com/rooms/953595,20160104002432,2016-01-04,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,none,"Queen Anne is a wonderful, truly functional vi...",...,10.0,f,NaN,WASHINGTON,f,strict,t,t,6,1.48
2,3308979,https://www.airbnb.com/rooms/3308979,20160104002432,2016-01-04,New Modern House-Amazing water view,New modern house built in 2013. Spectacular s...,"Our house is modern, light and fresh with a wa...",New modern house built in 2013. Spectacular s...,none,Upper Queen Anne is a charming neighborhood fu...,...,10.0,f,NaN,WASHINGTON,f,strict,f,f,2,1.15
3,7421966,https://www.airbnb.com/rooms/7421966,20160104002432,2016-01-04,Queen Anne Chateau,A charming apartment that sits atop Queen Anne...,NaN,A charming apartment that sits atop Queen Anne...,none,NaN,...,NaN,f,NaN,WASHINGTON,f,flexible,f,f,1,NaN
4,278830,https://www.airbnb.com/rooms/278830,20160104002432,2016-01-04,Charming craftsman 3 bdm house,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,none,We are in the beautiful neighborhood of Queen ...,...,9.0,f,NaN,WASHINGTON,f,strict,f,f,1,0.89


In [11]:
listings.shape

(3818, 92)

I want to understand the columns with missing values and which ones have the most percentage of missing values.
I sum up the msising values, create a new frame, and order by the percentage of missing values below.

In [12]:
null_listings = listings.isnull().sum()/(listings.isnull().sum() + listings.notnull().sum())
null_listings = null_listings.to_frame()
null_listings['percentage'] = null_listings[0]
null_listings= null_listings.drop(columns=[0])
null_listings = null_listings.loc[null_listings['percentage'] > 0]
null_listings.sort_values(by='percentage', ascending = False)

,percentage
license,1.000000
square_feet,0.974594
monthly_price,0.602672
security_deposit,0.511262
weekly_price,0.473808
notes,0.420639
neighborhood_overview,0.270299
cleaning_fee,0.269775
transit,0.244631
host_about,0.224987


These are all our missing variables.  I'm making the call to remove license and square feet since so many are missing!  I do think square feet would be an important variable, but there is just too much missing data to include it unfortunately!

In [13]:
listings= listings.drop(columns =['license', 'square_feet'])
listings.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_location,review_scores_value,requires_license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,241032,https://www.airbnb.com/rooms/241032,20160104002432,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,...,9.0,10.0,f,WASHINGTON,f,moderate,f,f,2,4.07
1,953595,https://www.airbnb.com/rooms/953595,20160104002432,2016-01-04,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,none,"Queen Anne is a wonderful, truly functional vi...",...,10.0,10.0,f,WASHINGTON,f,strict,t,t,6,1.48
2,3308979,https://www.airbnb.com/rooms/3308979,20160104002432,2016-01-04,New Modern House-Amazing water view,New modern house built in 2013. Spectacular s...,"Our house is modern, light and fresh with a wa...",New modern house built in 2013. Spectacular s...,none,Upper Queen Anne is a charming neighborhood fu...,...,10.0,10.0,f,WASHINGTON,f,strict,f,f,2,1.15
3,7421966,https://www.airbnb.com/rooms/7421966,20160104002432,2016-01-04,Queen Anne Chateau,A charming apartment that sits atop Queen Anne...,NaN,A charming apartment that sits atop Queen Anne...,none,NaN,...,NaN,NaN,f,WASHINGTON,f,flexible,f,f,1,NaN
4,278830,https://www.airbnb.com/rooms/278830,20160104002432,2016-01-04,Charming craftsman 3 bdm house,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,none,We are in the beautiful neighborhood of Queen ...,...,9.0,9.0,f,WASHINGTON,f,strict,f,f,1,0.89


Next I would like to join the new calendar dataframe, with the listing dataframe!  First, I must make sure the columns have a match column with the same name, so I rename the id column in listings to "listing_id" to match the ind_listing "listing_id" column.

In [14]:
listings['listing_id'] = listings['id']

In [15]:
combined_df = ind_listing.merge(listings, how='left', on ='listing_id')
combined_df

,listing_id,price_x,id,listing_url,scrape_id,last_scraped,name,summary,space,description,...,review_scores_location,review_scores_value,requires_license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,3335,101.589041,3335,https://www.airbnb.com/rooms/3335,20160104002432,2016-01-04,Sweet Seattle Urban Homestead 2 Bdr,Welcome! If you stay here you will be living i...,Welcome! Come enjoy your time in Seattle at a...,Welcome! If you stay here you will be living i...,...,NaN,NaN,f,WASHINGTON,f,strict,f,f,4,NaN
1,4291,82.000000,4291,https://www.airbnb.com/rooms/4291,20160104002432,2016-01-04,Sunrise in Seattle Master Suite,NaN,"Located in Seattle, this is a spacious, clean...","Located in Seattle, this is a spacious, clean...",...,9.0,9.0,f,WASHINGTON,f,moderate,f,f,5,1.14
2,5682,45.668493,5682,https://www.airbnb.com/rooms/5682,20160104002432,2016-01-04,"Cozy Studio, min. to downtown -WiFi",The Cozy Studio is a perfect launchpad for you...,"Hello fellow travelers, Save some money and ha...",The Cozy Studio is a perfect launchpad for you...,...,9.0,10.0,f,WASHINGTON,t,strict,f,t,1,4.21
3,6606,92.849315,6606,https://www.airbnb.com/rooms/6606,20160104002432,2016-01-04,"Fab, private seattle urban cottage!",NaN,"Soo centrally located, this is a little house ...","Soo centrally located, this is a little house ...",...,10.0,9.0,f,WASHINGTON,f,strict,f,f,1,0.66
4,7369,12.342466,7369,https://www.airbnb.com/rooms/7369,20160104002432,2016-01-04,launchingpad/landingpad,contemporary condo on the western edge of pike...,spacious condo with all the amenities,contemporary condo on the western edge of pike...,...,10.0,9.0,f,WASHINGTON,f,flexible,f,f,1,0.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3813,10332096,40.000000,10332096,https://www.airbnb.com/rooms/10332096,20160104002432,2016-01-04,Room & bath in suburban N Seattle,"Includes use of public space with lots of art,...",NaN,"Includes use of public space with lots of art,...",...,NaN,NaN,f,WASHINGTON,f,flexible,f,f,1,NaN
3814,10334184,118.684932,10334184,https://www.airbnb.com/rooms/10334184,20160104002432,2016-01-04,Historic Capitol Hill Garden Apt.,Perfectly situated off Broadway in a quiet par...,"One of the Hill's last historic spaces, this s...",Perfectly situated off Broadway in a quiet par...,...,NaN,NaN,f,WASHINGTON,f,flexible,f,f,1,NaN
3815,10339144,0.000000,10339144,https://www.airbnb.com/rooms/10339144,20160104002432,2016-01-04,Studio in the heart of Capitol Hill,"This is a great 3 floor, old world charm build...","Built in 1926, this studio has Hardwood/tile f...","This is a great 3 floor, old world charm build...",...,NaN,NaN,f,WASHINGTON,f,flexible,f,f,1,NaN
3816,10339145,237.904110,10339145,https://www.airbnb.com/rooms/10339145,20160104002432,2016-01-04,West Seattle Beachfront Apartment,Beautifully updated 2 bedroom/2 bath entire ap...,Spacious in-city apartment with 2 bedrooms and...,Beautifully updated 2 bedroom/2 bath entire ap...,...,NaN,NaN,f,WASHINGTON,t,strict,f,f,1,NaN


We have matched correctly as we have exactly 3818 rows, so a match for each row with the calculated price field!

In [16]:
reviews = pd.read_csv('reviews.csv')
reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,7202016,38917982,2015-07-19,28943674,Bianca,Cute and cozy place. Perfect location to every...
1,7202016,39087409,2015-07-20,32440555,Frank,Kelly has a great room in a very central locat...
2,7202016,39820030,2015-07-26,37722850,Ian,"Very spacious apartment, and in a great neighb..."
3,7202016,40813543,2015-08-02,33671805,George,Close to Seattle Center and all it has to offe...
4,7202016,41986501,2015-08-10,34959538,Ming,Kelly was a great host and very accommodating ...


In [17]:
reviews.shape

(84849, 6)

In [18]:
reviews.isnull().sum()

listing_id        0
id                0
date              0
reviewer_id       0
reviewer_name     0
comments         18
dtype: int64

In [19]:
condensed = reviews.groupby('listing_id')['comments'].sum()
condensed.to_frame()

,comments
listing_id,
4291,"We had a great stay in Seattle, great location..."
5682,Very sweet outgoing host with a nice family an...
6606,"The Urban Cottage is comfortable, beautiful, f..."
7369,I was staying with Shireen for a weekend and m...
9419,"If you love art, animals, and yoga, this is th..."
...,...
9995551,Awesome host! The cottage is nice and clean an...
10012724,This is a prime location of walking distance t...
10020221,Awesome room! Thanks for the kindness and lett...


In [20]:
total_df = combined_df.merge(condensed, how='left', on ='listing_id')
total_df.shape

(3818, 93)

In [21]:
total_df.isnull().sum()

listing_id                            0
price_x                               0
id                                    0
listing_url                           0
scrape_id                             0
                                   ... 
require_guest_profile_picture         0
require_guest_phone_verification      0
calculated_host_listings_count        0
reviews_per_month                   627
comments                            627
Length: 93, dtype: int64

In [22]:
total_df = total_df.fillna(0)

In [23]:
total_df.isnull().sum()

listing_id                          0
price_x                             0
id                                  0
listing_url                         0
scrape_id                           0
                                   ..
require_guest_profile_picture       0
require_guest_phone_verification    0
calculated_host_listings_count      0
reviews_per_month                   0
comments                            0
Length: 93, dtype: int64

In [24]:
price_matrix = total_df[['listing_id', 'price_x', 'price_y', 'weekly_price', 'monthly_price']]
price_matrix

,listing_id,price_x,price_y,weekly_price,monthly_price
0,3335,101.589041,$120.00,$550.00,"$1,600.00"
1,4291,82.000000,$82.00,$525.00,"$1,925.00"
2,5682,45.668493,$48.00,$375.00,"$1,250.00"
3,6606,92.849315,$90.00,$670.00,0
4,7369,12.342466,$85.00,0,0
...,...,...,...,...,...
3813,10332096,40.000000,$40.00,0,0
3814,10334184,118.684932,$120.00,0,0
3815,10339144,0.000000,$60.00,$400.00,0
3816,10339145,237.904110,$219.00,0,0


In [25]:
price_matrix['price_x'].value_counts()

0.000000      95
150.000000    24
100.000000    21
65.000000     19
75.000000     19
              ..
78.438356      1
0.684932       1
133.821918     1
103.145205     1
42.175342      1
Name: price_x, Length: 3113, dtype: int64

It looks like the price variable we were given "price_y" in the listing dataset will be of more use.  Using the mean price from "calendar" gave us 95 values with 0 price, whereas the "price_y" has a price for each datapoint.

In [26]:
total_df.isnull().sum()

listing_id                          0
price_x                             0
id                                  0
listing_url                         0
scrape_id                           0
                                   ..
require_guest_profile_picture       0
require_guest_phone_verification    0
calculated_host_listings_count      0
reviews_per_month                   0
comments                            0
Length: 93, dtype: int64

Confirming we no longer have any null values

Given the nature of hte problem, and that NLP is a bit above this assignment, I will stick to the listing dataset for further analysis.

In [27]:
listings.isnull().sum()

id                                    0
listing_url                           0
scrape_id                             0
last_scraped                          0
name                                  0
                                   ... 
require_guest_profile_picture         0
require_guest_phone_verification      0
calculated_host_listings_count        0
reviews_per_month                   627
listing_id                            0
Length: 91, dtype: int64

In [28]:
listings=listings.fillna(0)

In [29]:
listings_categories = listings.select_dtypes(include=['object'])
listings_categories.head()

,listing_url,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,...,has_availability,calendar_last_scraped,first_review,last_review,requires_license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification
0,https://www.airbnb.com/rooms/241032,2016-01-04,Stylish Queen Anne Apartment,0,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,0,0,0,...,t,2016-01-04,2011-11-01,2016-01-02,f,WASHINGTON,f,moderate,f,f
1,https://www.airbnb.com/rooms/953595,2016-01-04,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,none,"Queen Anne is a wonderful, truly functional vi...",What's up with the free pillows? Our home was...,"Convenient bus stops are just down the block, ...",...,t,2016-01-04,2013-08-19,2015-12-29,f,WASHINGTON,f,strict,t,t
2,https://www.airbnb.com/rooms/3308979,2016-01-04,New Modern House-Amazing water view,New modern house built in 2013. Spectacular s...,"Our house is modern, light and fresh with a wa...",New modern house built in 2013. Spectacular s...,none,Upper Queen Anne is a charming neighborhood fu...,Our house is located just 5 short blocks to To...,A bus stop is just 2 blocks away. Easy bus a...,...,t,2016-01-04,2014-07-30,2015-09-03,f,WASHINGTON,f,strict,f,f
3,https://www.airbnb.com/rooms/7421966,2016-01-04,Queen Anne Chateau,A charming apartment that sits atop Queen Anne...,0,A charming apartment that sits atop Queen Anne...,none,0,0,0,...,t,2016-01-04,0,0,f,WASHINGTON,f,flexible,f,f
4,https://www.airbnb.com/rooms/278830,2016-01-04,Charming craftsman 3 bdm house,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,none,We are in the beautiful neighborhood of Queen ...,Belltown,The nearest public transit bus (D Line) is 2 b...,...,t,2016-01-04,2012-07-10,2015-10-24,f,WASHINGTON,f,strict,f,f


In [30]:
listings_categories.columns

Index(['listing_url', 'last_scraped', 'name', 'summary', 'space',
       'description', 'experiences_offered', 'neighborhood_overview', 'notes',
       'transit', 'thumbnail_url', 'medium_url', 'picture_url',
       'xl_picture_url', 'host_url', 'host_name', 'host_since',
       'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'street', 'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'is_location_exact',
       'property_type', 'room_type', 'bed_type', 'amenities', 'price',
       'weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee',
       'extra_people', 'calendar_updated', 'has_availability',
       'calendar_last_s

In [31]:
new_list_cats = listings_categories[['experiences_offered', 'host_since',  'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
                                     'host_has_profile_pic', 'host_identity_verified','market',
       'smart_location', 'country_code', 'country', 'is_location_exact',
       'property_type', 'room_type', 'bed_type', 'amenities', 'price',
       'weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee','requires_license',  'instant_bookable',
       'cancellation_policy', 'require_guest_profile_picture',
       'require_guest_phone_verification',
       'extra_people', 'calendar_updated', 'has_availability']]
new_list_cats

,experiences_offered,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_has_profile_pic,host_identity_verified,market,smart_location,...,security_deposit,cleaning_fee,requires_license,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,extra_people,calendar_updated,has_availability
0,none,2011-08-11,within a few hours,96%,100%,f,t,t,Seattle,"Seattle, WA",...,0,0,f,f,moderate,f,f,$5.00,4 weeks ago,t
1,none,2013-02-21,within an hour,98%,100%,t,t,t,Seattle,"Seattle, WA",...,$100.00,$40.00,f,f,strict,t,t,$0.00,today,t
2,none,2014-06-12,within a few hours,67%,100%,f,t,t,Seattle,"Seattle, WA",...,"$1,000.00",$300.00,f,f,strict,f,f,$25.00,5 weeks ago,t
3,none,2013-11-06,0,0,0,f,t,t,Seattle,"Seattle, WA",...,0,0,f,f,flexible,f,f,$0.00,6 months ago,t
4,none,2011-11-29,within an hour,100%,0,f,t,t,Seattle,"Seattle, WA",...,$700.00,$125.00,f,f,strict,f,f,$15.00,7 weeks ago,t
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3813,none,2015-04-13,within a few hours,99%,100%,f,t,t,Seattle,"Seattle, WA",...,0,$230.00,f,f,strict,f,f,$0.00,today,t
3814,none,2015-10-14,within an hour,100%,100%,f,t,t,Seattle,"Seattle, WA",...,$500.00,$50.00,f,f,moderate,f,f,$25.00,2 days ago,t
3815,none,2015-12-30,0,0,0,f,t,f,Seattle,"Seattle, WA",...,$250.00,$35.00,f,f,moderate,f,f,$20.00,4 days ago,t
3816,none,2015-01-03,within an hour,100%,0,f,t,t,Seattle,"Seattle, WA",...,$300.00,$45.00,f,f,moderate,f,f,$0.00,never,t


In [32]:
new_list_cats['experiences_offered'].value_counts()

none    3818
Name: experiences_offered, dtype: int64

In [33]:
new_list_cats['host_since'] = pd.to_datetime(new_list_cats['host_since'])
new_list_cats['host_since']

0      2011-08-11
1      2013-02-21
2      2014-06-12
3      2013-11-06
4      2011-11-29
          ...    
3813   2015-04-13
3814   2015-10-14
3815   2015-12-30
3816   2015-01-03
3817   2014-04-25
Name: host_since, Length: 3818, dtype: datetime64[ns]

In [34]:
new_list_cats['host_since'] =pd.Timestamp.now().normalize() - pd.to_datetime( new_list_cats['host_since']) 


In [35]:
new_list_cats['host_since']

0      4364 days
1      3804 days
2      3328 days
3      3546 days
4      4254 days
          ...   
3813   3023 days
3814   2839 days
3815   2762 days
3816   3123 days
3817   3376 days
Name: host_since, Length: 3818, dtype: timedelta64[ns]

In [36]:
new_list_cats['host_response_time'].value_counts()

within an hour        1692
within a few hours     968
within a day           597
0                      523
a few days or more      38
Name: host_response_time, dtype: int64

In [37]:
new_list_cats['host_response_rate'] = new_list_cats['host_response_rate'].str.replace('%', '', regex=True)
new_list_cats['host_response_rate'] = new_list_cats['host_response_rate'].fillna(0)
new_list_cats['host_response_rate']

0        96
1        98
2        67
3         0
4       100
       ... 
3813     99
3814    100
3815      0
3816    100
3817    100
Name: host_response_rate, Length: 3818, dtype: object

In [38]:
new_list_cats['host_acceptance_rate'] = new_list_cats['host_acceptance_rate'].str.replace('%', '', regex=True)
new_list_cats['host_acceptance_rate'].value_counts()

100    3044
0         1
Name: host_acceptance_rate, dtype: int64

In [39]:
new_list_cats

,experiences_offered,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_has_profile_pic,host_identity_verified,market,smart_location,...,security_deposit,cleaning_fee,requires_license,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,extra_people,calendar_updated,has_availability
0,none,4364 days,within a few hours,96,100,f,t,t,Seattle,"Seattle, WA",...,0,0,f,f,moderate,f,f,$5.00,4 weeks ago,t
1,none,3804 days,within an hour,98,100,t,t,t,Seattle,"Seattle, WA",...,$100.00,$40.00,f,f,strict,t,t,$0.00,today,t
2,none,3328 days,within a few hours,67,100,f,t,t,Seattle,"Seattle, WA",...,"$1,000.00",$300.00,f,f,strict,f,f,$25.00,5 weeks ago,t
3,none,3546 days,0,0,NaN,f,t,t,Seattle,"Seattle, WA",...,0,0,f,f,flexible,f,f,$0.00,6 months ago,t
4,none,4254 days,within an hour,100,NaN,f,t,t,Seattle,"Seattle, WA",...,$700.00,$125.00,f,f,strict,f,f,$15.00,7 weeks ago,t
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3813,none,3023 days,within a few hours,99,100,f,t,t,Seattle,"Seattle, WA",...,0,$230.00,f,f,strict,f,f,$0.00,today,t
3814,none,2839 days,within an hour,100,100,f,t,t,Seattle,"Seattle, WA",...,$500.00,$50.00,f,f,moderate,f,f,$25.00,2 days ago,t
3815,none,2762 days,0,0,NaN,f,t,f,Seattle,"Seattle, WA",...,$250.00,$35.00,f,f,moderate,f,f,$20.00,4 days ago,t
3816,none,3123 days,within an hour,100,NaN,f,t,t,Seattle,"Seattle, WA",...,$300.00,$45.00,f,f,moderate,f,f,$0.00,never,t


Given the distribution of this variable being almost 100% the whole time, I'm going to drop it!

In [40]:
new_list_cats['cancellation_policy'].value_counts()

strict      1417
moderate    1251
flexible    1150
Name: cancellation_policy, dtype: int64

Next I want to change the "t" and "f" answers across the dataframe to 1 and 0s.

today            706
2 weeks ago      347
3 weeks ago      300
2 months ago     281
a week ago       248
yesterday        231
3 months ago     218
4 weeks ago      213
3 days ago       162
5 weeks ago      156
4 days ago       145
4 months ago     133
5 days ago       132
6 weeks ago      118
2 days ago        96
5 months ago      91
7 weeks ago       63
6 months ago      29
6 days ago        28
1 week ago        26
never             24
7 months ago      23
8 months ago      14
10 months ago      8
9 months ago       7
11 months ago      5
12 months ago      5
22 months ago      2
13 months ago      2
30 months ago      1
14 months ago      1
16 months ago      1
15 months ago      1
17 months ago      1
Name: calendar_updated, dtype: int64

In [44]:
cat_cols_lst = new_list_cats.columns

def create_dummy_df(df, cat_cols, dummy_na):
    '''
    INPUT:
    df - pandas dataframe with categorical variables you want to dummy
    cat_cols - list of strings that are associated with names of the categorical columns
    dummy_na - Bool holding whether you want to dummy NA vals of categorical columns or not
    
    OUTPUT:
    df - a new dataframe that has the following characteristics:
            1. contains all columns that were not specified as categorical
            2. removes all the original columns in cat_cols
            3. dummy columns for each of the categorical columns in cat_cols
            4. if dummy_na is True - it also contains dummy columns for the NaN values
            5. Use a prefix of the column name with an underscore (_) for separating 
    '''
    for col in  cat_cols:
        try:
            # for each cat add dummy var, drop original column
            df = pd.concat([df.drop(col, axis=1), pd.get_dummies(df[col], prefix=col, prefix_sep='_', drop_first=True, dummy_na=dummy_na)], axis=1)
        except:
            continue
    return df

In [49]:
df_new =create_dummy_df(new_list_cats, cat_cols_lst, dummy_na= False)